In [176]:
#from tiingo import TiingoClient
import pandas as pd
from datetime import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross, SignalStrategy
from backtesting.test import SMA, GOOG
import requests
#client = TiingoClient({'api_key': '345fbfca232ab5f373b5d890a7dadd3fafe30112'})

In [183]:
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=240&since=1522030972') 

In [184]:
resp.json()
dat = resp.json()['result']
data = {}
#Using this to format the data into a pandas dataframe

In [185]:
for i in range(len(dat['XXBTZUSD'])):
    data[dat['XXBTZUSD'][i][0]] = dat['XXBTZUSD'][i][1:]


In [186]:
dates = [datetime.fromtimestamp(key) for key in data.keys()]

In [187]:
ohlc_data = [value[:6] for value in data.values()]

In [188]:
ohlc_dict = {'Open': [float(val[0]) for val in ohlc_data],
             'High': [float(val[1]) for val in ohlc_data],
             'Low': [float(val[2]) for val in ohlc_data],
             'Close': [float(val[3]) for val in ohlc_data],
             'Volume': [float(val[5]) for val in ohlc_data]}

In [189]:
df = pd.DataFrame(ohlc_dict, index=dates)

In [190]:
df = (df / 1e6).assign(Volume=df.Volume * 1e6)
#Because backtesting.py is made for stocks, It doesn't support buying incremental values of BTC so to get around that we're just using price per Sat

In [11]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

#Unused for the cloud strategy, just using as reference

In [36]:
def maxminIch(n1, n2):
    line = (n1+n2)/2
    return line

These functions are calculating the average of period highs/lows for each line. 
I decided to use multiple functions over a single one (as you can see they all do the same thing) simply for better readability in the strat definition.

In [80]:
def Tenkan(n1, n2):
    line = (n1+n2)/2
    return line

In [81]:
def Kijun(n1, n2):
    line = (n1+n2)/2
    return line

In [73]:
def addshift(s1, window):
    shifted = s1.shift(window)
    return shifted

In [100]:
def senkouA(s1, window):
    shifted = s1.shift(window).fillna(0)
    return shifted

In [98]:
def senkouB(s1, window):
    shifted = s1.shift(window).fillna(0)
    return shifted

In [131]:
t = maxminIch(df['High'].rolling(20).max(), df['Low'].rolling(20).min())
k = maxminIch(df['High'].rolling(60).max(), df['Low'].rolling(60).min())
senkouB(((df['High'].rolling(20).max() + df['Low'].rolling(20).min()) / 2), 30)
#((df['High'].rolling(20).max() + df['Low'].rolling(20).min()) / 2)

#T and K are rolling averages of the period high and lows

2022-04-07 19:00:00    0.000000
2022-04-08 19:00:00    0.000000
2022-04-09 19:00:00    0.000000
2022-04-10 19:00:00    0.000000
2022-04-11 19:00:00    0.000000
                         ...   
2024-03-22 19:00:00    0.047612
2024-03-23 19:00:00    0.047612
2024-03-24 19:00:00    0.047621
2024-03-25 19:00:00    0.047761
2024-03-26 19:00:00    0.048842
Length: 720, dtype: float64

In [178]:
class IchimokuCloud(Strategy): #This should be a signal strategy - signals equal to actionable cloud patterns, set entry then pass to set signal
    nt = 20
    nk = 60
    nsb = 120
    nc = 30
    
    def init(self):
        close=self.data.Close
        self.tenkan = self.I(Tenkan, self.data['High'].s.rolling(self.nt).max(), self.data['Low'].s.rolling(self.nt).min())
        self.kijun = self.I(Kijun, self.data['High'].s.rolling(self.nk).max(), self.data['Low'].s.rolling(self.nk).min())
        self.senkouA = self.I(senkouA,
                              (maxminIch(self.data['High'].s.rolling(self.nt).max(), self.data['Low'].s.rolling(self.nt).min()) + maxminIch(self.data['High'].s.rolling(self.nk).max(), self.data['Low'].s.rolling(self.nk).min()))/2,
                              self.nc)
        self.senkouB = self.I(senkouB, maxminIch(self.data['High'].s.rolling(self.nsb).max(), self.data['Low'].s.rolling(self.nsb).min()), self.nc)
        self.lagging = self.I(addshift, close.s, (self.nc * -1))
        
    def next(self): #Kumo twist: If Senkou B > Senkou A and next-price > SB, Buy
        if (crossover(self.data.Close, self.senkouB) and self.senkouB > self.senkouA): 
            self.buy()
        elif (cross(self.tenkan, self.kijun) and self.kijun > self.tenkan):
            self.sell()

These frames are the cloud strategy. I haven't delved too deep in Backtesting.py so they're fairly simple. It probably wouldn't be difficult to flesh these out into a full strategy. Definitley keep model overfitting in your mind when testing strategies. Here's a great resource to read about it if you're interested: https://www.davidhbailey.com/dhbtalks/battle-quants.pdf

In [177]:
class IchimokuSignals(SignalStrategy):
    nt = 20
    nk = 60
    nsb = 120
    nc = 30
    
    def init(self):
        super().init()
        #precompute cloud
        close=self.data.Close
        self.tenkan = self.I(Tenkan, self.data['High'].s.rolling(self.nt).max(), self.data['Low'].s.rolling(self.nt).min())
        self.kijun = self.I(Kijun, self.data['High'].s.rolling(self.nk).max(), self.data['Low'].s.rolling(self.nk).min())
        self.senkouA = self.I(senkouA,
                              (maxminIch(self.data['High'].s.rolling(self.nt).max(), self.data['Low'].s.rolling(self.nt).min()) + maxminIch(self.data['High'].s.rolling(self.nk).max(), self.data['Low'].s.rolling(self.nk).min()))/2,
                              self.nc)
        self.senkouB = self.I(senkouB, maxminIch(self.data['High'].s.rolling(self.nsb).max(), self.data['Low'].s.rolling(self.nsb).min()), self.nc)
        self.lagging = self.I(addshift, close.s, (self.nc * -1))
        #establish signal points:
        signal = crossover(self.data.Close, self.senkouB) and self.senkouB > self.senkouA #Simple cloud cross strategy
        signal = signal

In [191]:
bt = Backtest(df, IchimokuCloud,
              cash=100, commission=.002,
              exclusive_orders=True, margin=0.5)

This function starts the backtest, values can be anything you want. Exclusive orders just means you wont open new positions if you're already in one.

In [192]:
output = bt.run()
bt.plot()

/Users/iowapsychopath/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/iowapsychopath/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p7587', ...)

In [193]:
output['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-4847,71,135,0.041262,0.042450,-5.757697,-0.028789,2023-12-17 18:00:00,2023-12-28 10:00:00,10 days 16:00:00
1,-4449,135,371,0.042365,0.042650,-1.269296,-0.006734,2023-12-28 10:00:00,2024-02-05 18:00:00,39 days 08:00:00
2,4351,371,482,0.042735,0.051094,36.366963,0.195583,2024-02-05 18:00:00,2024-02-24 06:00:00,18 days 12:00:00
3,-5073,482,659,0.050991,0.067199,-82.221597,-0.317851,2024-02-24 06:00:00,2024-03-24 19:00:00,29 days 13:00:00
4,1399,659,715,0.067333,0.066186,-1.605210,-0.017041,2024-03-24 19:00:00,2024-04-03 03:00:00,9 days 08:00:00
5,-1378,715,719,0.066054,0.065991,0.085888,0.000944,2024-04-03 03:00:00,2024-04-03 19:00:00,0 days 16:00:00


In [194]:
output

Start                     2023-12-05 22:00:00
End                       2024-04-03 19:00:00
Duration                    119 days 21:00:00
Exposure Time [%]                   90.138889
Equity Final [$]                    45.599051
Equity Peak [$]                    135.038475
Return [%]                         -54.400949
Buy & Hold Return [%]               51.113324
Return (Ann.) [%]                  -90.640981
Volatility (Ann.) [%]             1849.232157
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -88.579352
Avg. Drawdown [%]                  -12.638636
Max. Drawdown Duration       44 days 21:00:00
Avg. Drawdown Duration        8 days 21:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                      19.558304
Worst Trade [%]                    -31.785131
Avg. Trade [%]                    

In [118]:
output['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,467997,421,709,0.042735,0.067199,11448.985009,0.572449,2024-02-05 18:00:00,2024-03-24 19:00:00,48 days 01:00:00
1,637097,709,719,0.067333,0.070275,1874.276811,0.043692,2024-03-24 19:00:00,2024-03-26 11:00:00,1 days 16:00:00


In [146]:
output = bt.optimize(nt=range(5, 30, 5),
                     nk=range(10, 70, 5),
                     nsb=range(50, 200, 5),
                     maximize='Equity Final [$]',
                     constraint=lambda param: param.nt < param.nk < param.nsb, return_heatmap=True)

plot_heatmaps(output)

/Users/iowapsychopath/anaconda3/lib/python3.11/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1450 configurations.
  output = _optimize_grid()
/Users/iowapsychopath/anaconda3/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: heatmap must be heatmap Series as returned by `Backtest.optimize(..., return_heatmap=True)`